In [1]:
pip install nba_api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.3/257.3 kB 4.3 MB/s eta 0:00:00


In [24]:
from nba_api.stats.endpoints import shotchartdetail, playercareerstats, playerdashboardbyyearoveryear, playerdashboardbygamesplits,playercareerstats
from nba_api.stats.static import players, teams
import json
import requests
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [37]:
from nba_api.stats.library.parameters import PerMode36
class NbaScraper:
    """ Class to scrape data from the NBA official website.
    """
    @staticmethod
    def get_json_from_name(name: str, is_player=True) -> int:
        """ Get the json of a player or team from his name
        """
        from nba_api.stats.static import players, teams
        if is_player:
            nba_players = players.get_players()
            return [player for player in nba_players
                    if player['full_name'] == name][0]
        else:
            nba_teams = teams.get_teams()
            return [team for team in nba_teams
                    if team['full_name'] == name][0]

    @staticmethod
    def get_player_career(player_id: int) -> list:
        """ Get the career of a player from his id
        """
        from nba_api.stats.endpoints import playercareerstats
        career = playercareerstats.PlayerCareerStats(player_id=player_id)
        return career.get_data_frames()[0]

    @staticmethod
    def get_shot_data(id: int, team_ids: list, seasons: list) -> list:
        """ Get the shot data of a player from his id and seasons
        """
        from nba_api.stats.endpoints import shotchartdetail
        df = pd.DataFrame()
        for season in seasons:
            for team in team_ids:
                shot_data = shotchartdetail.ShotChartDetail(
                    team_id=team,
                    player_id=id,
                    context_measure_simple='FGA',
                    season_nullable=season
                )
                df = pd.concat([df, shot_data.get_data_frames()[0]])

        return df

    @staticmethod
    def get_all_ids(only_active=True) -> list:
        """ Get all the ids of the players
        """
        from nba_api.stats.static import players
        nba_players = players.get_players()
        if only_active:
            return [player['id'] for player in nba_players
                    if player['is_active']]
        return [player['id'] for player in nba_players]

    @staticmethod
    def getstats(player_id, per_mode36):
        # Create an instance of the PlayerCareerStats class
        player_career_stats = playercareerstats.PlayerCareerStats(player_id=player_id, per_mode36=per_mode)

        # Make the API request
        player_career_stats_data = player_career_stats.get_normalized_dict()

        return player_career_stats_data



In [58]:
import pandas as pd

# List of player IDs and their corresponding names
player_data = {
    201939: "Stephen Curry",
    202691: "Klay Thompson",
    203081: "Damian Lillard",
    2544: "LeBron James",
    1629673: "Jordan Poole",
    203999: "Nikola Jokic",
    201935: "James Harden",
    1627750: "Jamal Murray"
}

# List of player IDs
player_ids = list(player_data.keys())

# Replace per_mode with the desired value
per_mode = "Per36"

# Initialize an empty DataFrame to store the filtered data
filtered_data = pd.DataFrame()

# Loop through each player ID and fetch their statistics
for player_id in player_ids:
    player_stats = NbaScraper.getstats(player_id, per_mode)

    # Filter data for SEASON_ID '2022-2023'
    player_filtered_data = [player_data for player_data in player_stats['SeasonTotalsRegularSeason'] if player_data['SEASON_ID'] == '2022-23']

    # Create a DataFrame for the player's filtered data
    player_filtered_df = pd.DataFrame(player_filtered_data)

    # Add player ID as a column to identify the player
    player_filtered_df['PLAYER_ID'] = player_id

    # Add player name based on the dictionary
    player_filtered_df['Player_Name'] = player_data[player_id]

    # Append the player's filtered data to the overall DataFrame
    filtered_data = filtered_data.append(player_filtered_df, ignore_index=True)

# Print the filtered data
filtered_data.to_csv('player_stastdata.csv')


<ipython-input-58-40c90ad579ad>:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filtered_data = filtered_data.append(player_filtered_df, ignore_index=True)
<ipython-input-58-40c90ad579ad>:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filtered_data = filtered_data.append(player_filtered_df, ignore_index=True)
<ipython-input-58-40c90ad579ad>:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filtered_data = filtered_data.append(player_filtered_df, ignore_index=True)
<ipython-input-58-40c90ad579ad>:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filtered_data = filtered_data.append(player_filtered_df, ignore_index=True)
<ipython-inp

In [ ]:
names = ["Stephen Curry","Klay Thompson","Damian Lilard","LeBron James","Jordan Poole","Kyrie Irving","James Harden","Tray Young"]

In [ ]:
# get player id
name1 = "Stephen Curry"
id1 = NbaScraper.get_json_from_name(name1)['id']
print(id1)
career1 = NbaScraper.get_player_career(id1)
teams = ['GSW']
team_ids = list(set(career1[career1.TEAM_ABBREVIATION.isin(teams)].TEAM_ID.values))

seasons = ['2022-23']
shot_data1 = NbaScraper.get_shot_data(id1, team_ids, seasons)
shot_data1.tail()

201939


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
1128,Shot Chart Detail,0022201230,312,201939,Stephen Curry,1610612744,Golden State Warriors,2,1,3,...,Left Side Center(LC),24+ ft.,28,-172,223,1,0,20230409,POR,GSW
1129,Shot Chart Detail,0022201230,317,201939,Stephen Curry,1610612744,Golden State Warriors,2,0,32,...,Right Side Center(RC),24+ ft.,26,121,239,1,1,20230409,POR,GSW
1130,Shot Chart Detail,0022201230,342,201939,Stephen Curry,1610612744,Golden State Warriors,3,10,48,...,Center(C),Less Than 8 ft.,3,33,15,1,1,20230409,POR,GSW
1131,Shot Chart Detail,0022201230,353,201939,Stephen Curry,1610612744,Golden State Warriors,3,10,0,...,Left Side(L),24+ ft.,23,-225,79,1,1,20230409,POR,GSW
1132,Shot Chart Detail,0022201230,400,201939,Stephen Curry,1610612744,Golden State Warriors,3,6,27,...,Left Side(L),24+ ft.,23,-224,79,1,0,20230409,POR,GSW


In [ ]:
# get player id
name2 = "Klay Thompson"
id2 = NbaScraper.get_json_from_name(name2)['id']
print(id2)
career2 = NbaScraper.get_player_career(id2)
teams = ['GSW']
team_ids = list(set(career2[career2.TEAM_ABBREVIATION.isin(teams)].TEAM_ID.values))

seasons = ['2022-23']
shot_data2 = NbaScraper.get_shot_data(id2, team_ids, seasons)
shot_data2.tail()

202691


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
1247,Shot Chart Detail,0022201230,178,202691,Klay Thompson,1610612744,Golden State Warriors,2,11,29,...,Right Side Center(RC),24+ ft.,26,138,231,1,0,20230409,POR,GSW
1248,Shot Chart Detail,0022201230,223,202691,Klay Thompson,1610612744,Golden State Warriors,2,8,10,...,Right Side(R),24+ ft.,22,229,20,1,0,20230409,POR,GSW
1249,Shot Chart Detail,0022201230,348,202691,Klay Thompson,1610612744,Golden State Warriors,3,10,9,...,Center(C),Less Than 8 ft.,6,-50,37,1,0,20230409,POR,GSW
1250,Shot Chart Detail,0022201230,360,202691,Klay Thompson,1610612744,Golden State Warriors,3,9,13,...,Right Side(R),8-16 ft.,14,141,-7,1,0,20230409,POR,GSW
1251,Shot Chart Detail,0022201230,392,202691,Klay Thompson,1610612744,Golden State Warriors,3,6,58,...,Left Side(L),24+ ft.,23,-235,15,1,1,20230409,POR,GSW


In [ ]:
# get player id
name3 = "Damian Lillard"
id3 = NbaScraper.get_json_from_name(name3)['id']
print(id3)
career3 = NbaScraper.get_player_career(id3)
teams = ['POR']
team_ids = list(set(career3[career3.TEAM_ABBREVIATION.isin(teams)].TEAM_ID.values))

seasons = ['2022-23']
shot_data3 = NbaScraper.get_shot_data(id3, team_ids, seasons)
shot_data3.tail()

203081


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
1197,Shot Chart Detail,0022201091,439,203081,Damian Lillard,1610612757,Portland Trail Blazers,3,4,26,...,Center(C),Less Than 8 ft.,5,40,34,1,1,20230322,UTA,POR
1198,Shot Chart Detail,0022201091,451,203081,Damian Lillard,1610612757,Portland Trail Blazers,3,3,33,...,Left Side Center(LC),24+ ft.,27,-121,243,1,0,20230322,UTA,POR
1199,Shot Chart Detail,0022201091,601,203081,Damian Lillard,1610612757,Portland Trail Blazers,4,5,51,...,Center(C),Less Than 8 ft.,5,-44,36,1,1,20230322,UTA,POR
1200,Shot Chart Detail,0022201091,636,203081,Damian Lillard,1610612757,Portland Trail Blazers,4,3,25,...,Left Side Center(LC),24+ ft.,29,-225,186,1,0,20230322,UTA,POR
1201,Shot Chart Detail,0022201091,661,203081,Damian Lillard,1610612757,Portland Trail Blazers,4,1,41,...,Left Side Center(LC),24+ ft.,25,-126,225,1,1,20230322,UTA,POR


In [ ]:
# get player id
name4 = "LeBron James"
id4 = NbaScraper.get_json_from_name(name4)['id']
print(id4)
career4 = NbaScraper.get_player_career(id4)
teams = ['LAL']
team_ids = list(set(career4[career4.TEAM_ABBREVIATION.isin(teams)].TEAM_ID.values))

seasons = ['2022-23']
shot_data4 = NbaScraper.get_shot_data(id4, team_ids, seasons)
shot_data4.tail()

2544


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
1214,Shot Chart Detail,0022201228,575,2544,LeBron James,1610612747,Los Angeles Lakers,4,3,32,...,Center(C),24+ ft.,30,-75,294,1,1,20230409,LAL,UTA
1215,Shot Chart Detail,0022201228,580,2544,LeBron James,1610612747,Los Angeles Lakers,4,2,55,...,Center(C),24+ ft.,29,60,284,1,0,20230409,LAL,UTA
1216,Shot Chart Detail,0022201228,594,2544,LeBron James,1610612747,Los Angeles Lakers,4,1,59,...,Center(C),Less Than 8 ft.,5,20,56,1,0,20230409,LAL,UTA
1217,Shot Chart Detail,0022201228,605,2544,LeBron James,1610612747,Los Angeles Lakers,4,1,7,...,Center(C),24+ ft.,25,-29,253,1,1,20230409,LAL,UTA
1218,Shot Chart Detail,0022201228,609,2544,LeBron James,1610612747,Los Angeles Lakers,4,0,39,...,Right Side Center(RC),24+ ft.,31,189,251,1,1,20230409,LAL,UTA


In [ ]:
# get player id
name5 = "Jordan Poole"
id5 = NbaScraper.get_json_from_name(name5)['id']
print(id5)
career5 = NbaScraper.get_player_career(id5)
teams = ['GSW']
team_ids = list(set(career5[career5.TEAM_ABBREVIATION.isin(teams)].TEAM_ID.values))

seasons = ['2022-23']
shot_data5 = NbaScraper.get_shot_data(id5, team_ids, seasons)
shot_data5.tail()

1629673


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
1273,Shot Chart Detail,0022201230,161,1629673,Jordan Poole,1610612744,Golden State Warriors,1,0,17,...,Right Side Center(RC),24+ ft.,25,134,217,1,1,20230409,POR,GSW
1274,Shot Chart Detail,0022201230,180,1629673,Jordan Poole,1610612744,Golden State Warriors,2,11,23,...,Center(C),Less Than 8 ft.,1,10,-3,1,1,20230409,POR,GSW
1275,Shot Chart Detail,0022201230,213,1629673,Jordan Poole,1610612744,Golden State Warriors,2,9,21,...,Center(C),Less Than 8 ft.,6,-55,42,1,1,20230409,POR,GSW
1276,Shot Chart Detail,0022201230,474,1629673,Jordan Poole,1610612744,Golden State Warriors,3,1,24,...,Center(C),Less Than 8 ft.,1,-17,10,1,0,20230409,POR,GSW
1277,Shot Chart Detail,0022201230,504,1629673,Jordan Poole,1610612744,Golden State Warriors,4,10,48,...,Center(C),Less Than 8 ft.,1,9,10,1,1,20230409,POR,GSW


In [ ]:
# get player id
name6 = "Nikola Jokic"
id6 = NbaScraper.get_json_from_name(name6)['id']
print(id6)
career6 = NbaScraper.get_player_career(id6)
teams = ['DEN']
team_ids = list(set(career6[career6.TEAM_ABBREVIATION.isin(teams)].TEAM_ID.values))

seasons = ['2022-23']
shot_data6 = NbaScraper.get_shot_data(id6, team_ids, seasons)
shot_data6.tail()

203999


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
1017,Shot Chart Detail,0022201213,96,203999,Nikola Jokic,1610612743,Denver Nuggets,1,4,47,...,Center(C),Less Than 8 ft.,5,11,56,1,0,20230408,UTA,DEN
1018,Shot Chart Detail,0022201213,287,203999,Nikola Jokic,1610612743,Denver Nuggets,2,3,25,...,Center(C),Less Than 8 ft.,7,17,69,1,0,20230408,UTA,DEN
1019,Shot Chart Detail,0022201213,321,203999,Nikola Jokic,1610612743,Denver Nuggets,2,0,47,...,Center(C),Less Than 8 ft.,2,20,-2,1,1,20230408,UTA,DEN
1020,Shot Chart Detail,0022201213,384,203999,Nikola Jokic,1610612743,Denver Nuggets,3,7,39,...,Center(C),Less Than 8 ft.,7,-11,73,1,1,20230408,UTA,DEN
1021,Shot Chart Detail,0022201213,406,203999,Nikola Jokic,1610612743,Denver Nuggets,3,5,59,...,Right Side Center(RC),24+ ft.,26,107,243,1,0,20230408,UTA,DEN


In [ ]:
# get player id
name7 = "James Harden"
id7 = NbaScraper.get_json_from_name(name7)['id']
print(id7)
career7 = NbaScraper.get_player_career(id7)
teams = ['PHI']
team_ids = list(set(career7[career7.TEAM_ABBREVIATION.isin(teams)].TEAM_ID.values))

seasons = ['2022-23']
shot_data7 = NbaScraper.get_shot_data(id7, team_ids, seasons)
shot_data7.tail()

201935


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
837,Shot Chart Detail,0022201198,260,201935,James Harden,1610612755,Philadelphia 76ers,2,4,18,...,Right Side Center(RC),24+ ft.,25,120,221,1,0,20230406,PHI,MIA
838,Shot Chart Detail,0022201198,276,201935,James Harden,1610612755,Philadelphia 76ers,2,2,6,...,Right Side Center(RC),24+ ft.,25,151,208,1,0,20230406,PHI,MIA
839,Shot Chart Detail,0022201198,287,201935,James Harden,1610612755,Philadelphia 76ers,2,0,35,...,Center(C),Less Than 8 ft.,6,-17,62,1,1,20230406,PHI,MIA
840,Shot Chart Detail,0022201198,316,201935,James Harden,1610612755,Philadelphia 76ers,3,9,49,...,Right Side(R),24+ ft.,23,233,55,1,1,20230406,PHI,MIA
841,Shot Chart Detail,0022201198,331,201935,James Harden,1610612755,Philadelphia 76ers,3,8,0,...,Left Side Center(LC),24+ ft.,24,-224,111,1,1,20230406,PHI,MIA


In [ ]:
# get player id
name8 = "Jamal Murray"
id8 = NbaScraper.get_json_from_name(name8)['id']
print(id8)
career8 = NbaScraper.get_player_career(id8)
teams = ['DEN']
team_ids = list(set(career8[career8.TEAM_ABBREVIATION.isin(teams)].TEAM_ID.values))

seasons = ['2022-23']
shot_data8 = NbaScraper.get_shot_data(id8, team_ids, seasons)
shot_data8.tail()

1627750


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
1036,Shot Chart Detail,0022201213,363,1627750,Jamal Murray,1610612743,Denver Nuggets,3,9,7,...,Center(C),Less Than 8 ft.,0,0,0,1,1,20230408,UTA,DEN
1037,Shot Chart Detail,0022201213,369,1627750,Jamal Murray,1610612743,Denver Nuggets,3,8,40,...,Left Side Center(LC),24+ ft.,26,-159,210,1,0,20230408,UTA,DEN
1038,Shot Chart Detail,0022201213,392,1627750,Jamal Murray,1610612743,Denver Nuggets,3,6,47,...,Right Side Center(RC),24+ ft.,25,89,244,1,1,20230408,UTA,DEN
1039,Shot Chart Detail,0022201213,478,1627750,Jamal Murray,1610612743,Denver Nuggets,3,1,10,...,Left Side Center(LC),24+ ft.,30,-186,242,1,1,20230408,UTA,DEN
1040,Shot Chart Detail,0022201213,498,1627750,Jamal Murray,1610612743,Denver Nuggets,3,0,1,...,Center(C),24+ ft.,24,23,248,1,0,20230408,UTA,DEN


In [ ]:
dataframes_to_concat = [shot_data1, shot_data2, shot_data3, shot_data4, shot_data5, shot_data6, shot_data7, shot_data8]

shot_data = pd.concat(dataframes_to_concat, ignore_index=True)
shot_data

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0022200002,61,201939,Stephen Curry,1610612744,Golden State Warriors,1,7,27,...,Center(C),Less Than 8 ft.,6,60,10,1,0,20221018,GSW,LAL
1,Shot Chart Detail,0022200002,64,201939,Stephen Curry,1610612744,Golden State Warriors,1,7,22,...,Right Side(R),16-24 ft.,16,150,79,1,1,20221018,GSW,LAL
2,Shot Chart Detail,0022200002,68,201939,Stephen Curry,1610612744,Golden State Warriors,1,7,11,...,Center(C),24+ ft.,27,7,273,1,0,20221018,GSW,LAL
3,Shot Chart Detail,0022200002,102,201939,Stephen Curry,1610612744,Golden State Warriors,1,5,16,...,Left Side Center(LC),24+ ft.,24,-151,197,1,0,20221018,GSW,LAL
4,Shot Chart Detail,0022200002,132,201939,Stephen Curry,1610612744,Golden State Warriors,1,3,52,...,Left Side Center(LC),24+ ft.,24,-81,229,1,0,20221018,GSW,LAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8984,Shot Chart Detail,0022201213,363,1627750,Jamal Murray,1610612743,Denver Nuggets,3,9,7,...,Center(C),Less Than 8 ft.,0,0,0,1,1,20230408,UTA,DEN
8985,Shot Chart Detail,0022201213,369,1627750,Jamal Murray,1610612743,Denver Nuggets,3,8,40,...,Left Side Center(LC),24+ ft.,26,-159,210,1,0,20230408,UTA,DEN
8986,Shot Chart Detail,0022201213,392,1627750,Jamal Murray,1610612743,Denver Nuggets,3,6,47,...,Right Side Center(RC),24+ ft.,25,89,244,1,1,20230408,UTA,DEN
8987,Shot Chart Detail,0022201213,478,1627750,Jamal Murray,1610612743,Denver Nuggets,3,1,10,...,Left Side Center(LC),24+ ft.,30,-186,242,1,1,20230408,UTA,DEN


In [ ]:
shot_data.to_csv('shot_data.csv')